# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,23.40,75,62,9.98,MU,1689748570
1,1,martapura,-3.4167,114.8500,32.05,66,40,3.60,ID,1689748570
2,2,lebu,-37.6167,-73.6500,11.36,96,100,5.96,CL,1689748570
3,3,port-aux-francais,-49.3500,70.2167,2.08,70,100,12.14,TF,1689748570
4,4,khandyga,62.6667,135.6000,23.38,51,96,1.38,RU,1689748570


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
city_map_1 = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      frame_width = 800,
                                      frame_height = 600,
                                      alpha = 0.75,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max_Temp"] < 27) 
                                       & (city_data_df["Max_Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] == 0)
                                       & (city_data_df["Wind_Speed"] < 4.5)]

# Drop any rows with null values
weather_filtered_df = weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
32,32,portland,45.5234,-122.6762,23.96,53,0,4.12,US,1689748249
39,39,bethel,41.3712,-73.4140,22.17,87,0,1.54,US,1689748575
139,139,belyy yar,53.6039,91.3903,26.94,44,0,2.00,RU,1689748593
156,156,fort bragg,35.1390,-79.0060,26.91,85,0,4.12,US,1689748528
160,160,marlton,39.8912,-74.9218,23.83,90,0,0.00,US,1689748456
359,359,roswell,34.0232,-84.3616,25.31,76,0,3.60,US,1689748585
368,368,show low,34.2542,-110.0298,23.97,57,0,2.57,US,1689748627
386,386,karasu,41.0710,30.7854,24.76,61,0,1.60,TR,1689748631
391,391,prien,30.1819,-93.2738,26.99,93,0,2.06,US,1689748632
441,441,brandon,27.9378,-82.2859,26.95,92,0,1.54,US,1689748640


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,portland,US,45.5234,-122.6762,53,
39,bethel,US,41.3712,-73.4140,87,
139,belyy yar,RU,53.6039,91.3903,44,
156,fort bragg,US,35.1390,-79.0060,85,
160,marlton,US,39.8912,-74.9218,90,
359,roswell,US,34.0232,-84.3616,76,
368,show low,US,34.2542,-110.0298,57,
386,karasu,TR,41.0710,30.7854,61,
391,prien,US,30.1819,-93.2738,93,
441,brandon,US,27.9378,-82.2859,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
portland - nearest hotel: the Hoxton
bethel - nearest hotel: Hampton Inn Danbury
belyy yar - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
marlton - nearest hotel: Hyatt Place Mt. Laurel
roswell - nearest hotel: Best Western Plus Hotel
show low - nearest hotel: Holiday Inn Express Showlow
karasu - nearest hotel: No hotel found
prien - nearest hotel: Golden Nugget Lake Charles Hotel & Casino
brandon - nearest hotel: Brandon Motor Lodge
ambovombe - nearest hotel: La source
freeport - nearest hotel: The Freeport Inn and Marina
jackson - nearest hotel: Brookside
dagomys - nearest hotel: Royal
tisul' - nearest hotel: No hotel found
noyemberyan - nearest hotel: Levon 2


,City,Country,Lat,Lng,Humidity,Hotel Name
32,portland,US,45.5234,-122.6762,53,the Hoxton
39,bethel,US,41.3712,-73.4140,87,Hampton Inn Danbury
139,belyy yar,RU,53.6039,91.3903,44,No hotel found
156,fort bragg,US,35.1390,-79.0060,85,Airborne Inn Lodging
160,marlton,US,39.8912,-74.9218,90,Hyatt Place Mt. Laurel
359,roswell,US,34.0232,-84.3616,76,Best Western Plus Hotel
368,show low,US,34.2542,-110.0298,57,Holiday Inn Express Showlow
386,karasu,TR,41.0710,30.7854,61,No hotel found
391,prien,US,30.1819,-93.2738,93,Golden Nugget Lake Charles Hotel & Casino
441,brandon,US,27.9378,-82.2859,92,Brandon Motor Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"])


hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)